In [4]:
!gdown 1O-yTEGs_us3hvnatQN0jIx6VqeZzmNq0

Downloading...
From: https://drive.google.com/uc?id=1O-yTEGs_us3hvnatQN0jIx6VqeZzmNq0
To: d:\Courses\AIO2024\New folder\AIO Learning\AIO2024\Module1\Week2\Thursday\aio-competition-part-1-housing-price-prediction.zip

  0%|          | 0.00/560k [00:00<?, ?B/s]
 94%|█████████▎| 524k/560k [00:00<00:00, 2.06MB/s]
100%|██████████| 560k/560k [00:00<00:00, 2.14MB/s]


In [6]:
import zipfile
with zipfile.ZipFile('aio-competition-part-1-housing-price-prediction.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [10]:
housing_data = pd.read_csv('train.csv')

In [11]:
housing_data.head()

,id,Area,Location,No. of Bedrooms,Resale,MaintenanceStaff,Gymnasium,SwimmingPool,LandscapedGardens,JoggingTrack,...,BED,VaastuCompliant,Microwave,GolfCourse,TV,DiningTable,Sofa,Wardrobe,Refrigerator,Price
0,26813,964,Hooghly Chinsurah,3,0,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,7500000
1,10365,1075,Virar West,2,1,1,1,0,0,0,...,1,0,0,0,0,0,0,0,0,6200000
2,17863,2092,Manikonda,3,0,1,1,1,1,0,...,0,1,0,0,0,0,0,0,0,10400000
3,24337,1174,Devanahalli,2,1,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,18600000
4,12590,1380,Kharghar,2,1,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9500000


In [12]:
housing_data = housing_data.drop(columns=["Resale"]) # Nhà mua lại hay không không ảnh hưởng tới giá
housing_data = housing_data.drop(columns=["Location"])
housing_data = housing_data.drop(columns=["id"])

In [13]:
housing_data.head()

,Area,No. of Bedrooms,MaintenanceStaff,Gymnasium,SwimmingPool,LandscapedGardens,JoggingTrack,RainWaterHarvesting,IndoorGames,ShoppingMall,...,BED,VaastuCompliant,Microwave,GolfCourse,TV,DiningTable,Sofa,Wardrobe,Refrigerator,Price
0,964,3,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,7500000
1,1075,2,1,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,6200000
2,2092,3,1,1,1,1,0,1,1,0,...,0,1,0,0,0,0,0,0,0,10400000
3,1174,2,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,18600000
4,1380,2,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9500000


In [14]:
def mix_feature(housing_data, train_data=True):
  # Kết hợp các biến số học
  housing_data['Area_per_Bedroom'] = housing_data['Area'] / housing_data['No. of Bedrooms']
  # Tạo các biến tương tác
  housing_data['Bedrooms_Gymnasium'] = housing_data['No. of Bedrooms'] * housing_data['Gymnasium']
  # Biến đổi log
  if train_data:
    housing_data['Log_Price'] = housing_data['Price'].apply(lambda x: np.log(x) if x > 0 else 0)
  # Biến đổi root
  housing_data['Root_Area'] = housing_data['Area'].apply(lambda x: np.sqrt(x))

  # Kết hợp các biến số học khác: MaintenanceStaff và CarParking
  housing_data['Staff_per_CarParking'] = housing_data['MaintenanceStaff'] / (housing_data['CarParking'] + 1)
  # Kết hợp các biến số học khác: SwimmingPool và Gymnasium
  housing_data['Pool_Gym'] = housing_data['SwimmingPool'] + housing_data['Gymnasium']
  # Kết hợp các biến số học khác: PowerBackup và 24X7Security
  housing_data['Power_Security'] = housing_data['PowerBackup'] + housing_data['24X7Security']
  # Tạo các biến tương tác khác: SportsFacility và ClubHouse
  housing_data['Sports_Club'] = housing_data['SportsFacility'] * housing_data['ClubHouse']
  # Tạo các biến tương tác khác: Intercom và VaastuCompliant
  housing_data['Intercom_Vaastu'] = housing_data['Intercom'] * housing_data['VaastuCompliant']

  # Kết hợp các biến số học khác: ATM và CarParking
  housing_data['ATM_per_CarParking'] = housing_data['ATM'] / (housing_data['CarParking'] + 1)
  # Kết hợp các biến số học khác: SportsFacility và SwimmingPool
  housing_data['Sports_Pool'] = housing_data['SportsFacility'] + housing_data['SwimmingPool']
  # Tạo các biến tương tác khác: ClubHouse và Gymnasium
  housing_data['Club_Gym'] = housing_data['ClubHouse'] * housing_data['Gymnasium']

  return housing_data

In [15]:
data_test = pd.read_csv('test.csv')

print(data_test.shape) # thiếu cột price
print(housing_data.shape) # thiếu cột resale do đã xóa

(6593, 40)
(26370, 38)


In [16]:
data_test = data_test.drop(columns=["Resale"]) # xóa resale ở dataset
data_test = data_test.drop(columns=["Location"]) # xóa location ở dataset
data_test = data_test.drop(columns=["id"]) # xóa location ở dataset
print(data_test.shape)

(6593, 37)


In [17]:
# mix feature
data_test = mix_feature(data_test, False)
housing_data = mix_feature(housing_data, True)
housing_data = housing_data.drop(columns=['Price']) # sử dụng log_price
print(housing_data.shape)
print(data_test.shape)

(26370, 49)
(6593, 48)


In [18]:
X = housing_data.drop(columns=['Log_Price'])  # Bỏ cột mục tiêu là log_price
y = housing_data['Log_Price']  # Lấy cột mục tiêu là log_price
print(X.shape)
print(data_test.shape)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

(26370, 48)
(6593, 48)


In [22]:
# data processing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
numeric_columns = X.select_dtypes(include=[np.number]).columns.tolist()

encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded_categorical_X = encoder.fit_transform(X[categorical_columns])
encoded_categorical_data_test = encoder.transform(data_test[categorical_columns])

encoded_categorical_df_train = pd.DataFrame(encoded_categorical_X, columns=encoder.get_feature_names_out(categorical_columns))
encoded_categorical_df_test = pd.DataFrame(encoded_categorical_data_test, columns=encoder.get_feature_names_out(categorical_columns))

data_train_encoded = pd.concat([X[numeric_columns], encoded_categorical_df_train], axis=1)
data_test_encoded = pd.concat([data_test[numeric_columns], encoded_categorical_df_test], axis=1)

scaler = StandardScaler()
scaled_data_train = scaler.fit_transform(data_train_encoded)
scaled_data_test = scaler.transform(data_test_encoded)

data_train_scaled = pd.DataFrame(scaled_data_train, columns=data_train_encoded.columns)
data_test_scaled = pd.DataFrame(scaled_data_test, columns=data_test_encoded.columns)


In [23]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
# Hàm lựa chọn biến quan trọng bằng Random Forest Feature Importance
def select_features_random_forest(X_train, y_train):
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    importances = rf.feature_importances_
    indices = np.argsort(importances)[::-1]
    return X_train.columns[indices], importances[indices]

In [25]:
# Lựa chọn biến quan trọng bằng Random Forest
important_features_rf, importances_rf = select_features_random_forest(X_train, y_train)

# Hiển thị kết quả Random Forest Feature Importance
rf_feature_importance = pd.DataFrame({'Feature': important_features_rf, 'Importance': importances_rf})
print(rf_feature_importance)

                 Feature  Importance
0                   Area    0.294344
1              Root_Area    0.289366
2       Area_per_Bedroom    0.237330
3       MaintenanceStaff    0.021169
4   Staff_per_CarParking    0.017705
5     Bedrooms_Gymnasium    0.013837
6     ATM_per_CarParking    0.013626
7          Gasconnection    0.011069
8          LiftAvailable    0.005384
9     Children'splayarea    0.004908
10              Intercom    0.004897
11       No. of Bedrooms    0.004574
12   RainWaterHarvesting    0.004518
13            CarParking    0.004250
14                   BED    0.003997
15        Power_Security    0.003961
16                  Wifi    0.003672
17     LandscapedGardens    0.003235
18       VaastuCompliant    0.003040
19              Pool_Gym    0.002916
20           Sports_Pool    0.002881
21          StaffQuarter    0.002845
22           IndoorGames    0.002646
23          JoggingTrack    0.002592
24             ClubHouse    0.002531
25      MultipurposeRoom    0.002423
2

In [26]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
# dùng các feature trên 0.002
num_rf_features = 48
important_features_rf_selected = important_features_rf[:num_rf_features]

X_train_selected = X_train[important_features_rf_selected]
X_test_selected = X_test[important_features_rf_selected]

# Huấn luyện mô hình Linear Regression
linear_regressor_rf = LinearRegression()
linear_regressor_rf.fit(X_train_selected, y_train)

# Dự đoán trên tập kiểm tra
y_pred = linear_regressor_rf.predict(X_test_selected)

# Tính toán RMSE # 0.7320508751497298
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'RMSE: {rmse}')

RMSE: 0.73205087514973


# Submit

In [27]:
data_test_selected = data_test[important_features_rf_selected]
y_test_pred = linear_regressor_rf.predict(data_test_selected)
y_test_pred[:10]

array([15.78670748, 16.13338336, 15.8263775 , 15.75664031, 15.28816604,
       15.92743661, 16.20758959, 15.71832537, 15.85365285, 15.94181678])

In [28]:
submit_df = pd.read_csv('test.csv')
list_id = submit_df['id']
list_id = np.array(list_id.to_list())
len(list_id)


6593

In [29]:
df = pd.DataFrame({'id': list_id, 'Price': y_test_pred})
df
df.to_csv('submission.csv', index=False)